<pre>
      _                 _              _____ 
     | |               | |            / __  \
  ___| |__   __ _ _ __ | |_ ___ _ __  `' / /'
 / __| '_ \ / _` | '_ \| __/ _ \ '__|   / /  
| (__| | | | (_| | |_) | ||  __/ |    ./ /___
 \___|_| |_|\__,_| .__/ \__\___|_|    \_____/
                 | |                         
                 |_|                         
</pre>

## Глава 2

- [2 Запросы SQL к связанным таблицам](#2-Запросы-SQL-к-связанным-таблицам)
    - [2.1 Связи между таблицами](#2.1-Связи-между-таблицами)
    - [2.2 Запросы на выборку, соединение таблиц](#2.2-Запросы-на-выборку,-соединение-таблиц)
    - [2.3 Запросы корректировки, соединение таблиц](#2.3-Запросы-корректировки,-соединение-таблиц)
    - [](#)
    - [](#)
    - [](#)



#  2 Запросы SQL к связанным таблицам
[`⌂`](#Глава-2)


<hr>

## 2.1 Связи между таблицами

[`⌂`](#Глава-2)

<hr>

### Связь «один ко многим»

[`⌂`](#Глава-2)

<hr>

### Связь «многие ко многим»

[`⌂`](#Глава-2)

<hr>

## 2.2 Запросы на выборку, соединение таблиц

[`⌂`](#Глава-2)

<hr>

## 2.3 Запросы корректировки, соединение таблиц

[`⌂`](#Глава-2)

<hr>

## Выборка данных по условию

[`⌂`](#Глава-2)